<a href="https://colab.research.google.com/github/Xalp/NLP1d/blob/master/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Xalp/NLP1d

Cloning into 'NLP1d'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 39 (delta 8), reused 20 (delta 1), pack-reused 0
Unpacking objects: 100% (39/39), done.


In [ ]:
import os
os.chdir("/content/NLP1d")

## Part 1

In [ ]:
# Emission probability
from collections import defaultdict
import numpy as np
def emission(train):
    '''
    x: str, word
    y: str, tag
    train: training set path
    '''
    count_y_x = defaultdict(float)
    count_y  = defaultdict(float)
    emission_probs = defaultdict(float)
    dict_f = defaultdict()
    with open(train) as f:
        for line in f.readlines():
            if line == '\n': continue
            x, y = line.split()
            count_y_x[(y,x)] += 1
    
    for (y,x), cnt in count_y_x.items():
        count_y[y] += cnt
    for (y,x) in count_y_x.keys():
        e_x_y = count_y_x[(y,x)]/count_y[y]
        emission_probs[(x,y)] = np.log(e_x_y)
        str_x_y = 'emission:'+str(y)+'+'+str(x)
        dict_f[str_x_y] = np.log(e_x_y)

    return emission_probs, dict_f


In [ ]:
import numpy as np
# Transition probability
def transition(train):
  with open(train,"r") as f:
    data = f.read().rstrip().splitlines()

  count_bigram = {}
  count_unigram = {}

  count_unigram["START"] = 0
  count_unigram["STOP"] = 0

  for i in range(len(data)):
    inst = data[i]
    # if this is an empty line:
    if len(inst) == 0:
      # building with previous inst
      tag = "STOP"
      if i != 0:
        prev_inst = data[i-1]
        prev_tag = prev_inst.split()[-1]
        if (tag, prev_tag) not in count_bigram.keys():
          count_bigram[(tag, prev_tag)] = 1
        else:
          count_bigram[(tag, prev_tag)] += 1
      # building with the next inst
      tag = "START"
      if i != len(data) - 1:
        next_inst = data[i+1]
        next_tag = next_inst.split()[-1]
        if (tag, next_tag) not in count_bigram.keys():
          count_bigram[(tag, next_tag)] = 1
        else:
          count_bigram[(tag, next_tag)] += 1
      # increase the count of start and stop
      count_unigram["START"] += 1
      count_unigram["STOP"] += 1
      continue
    # do in all cases:
    tag = inst.split()[-1]
    if tag not in count_unigram.keys():
      count_unigram[tag] = 1
    else:
      count_unigram[tag] += 1
    # if not the last inst:
    if i != len(data) - 1:
      next_inst = data[i+1]
      if len(next_inst) == 0:
        continue
      next_tag = next_inst.split()[-1]
      if (tag, next_tag) not in count_bigram.keys():
        count_bigram[(tag, next_tag)] = 1
      else:
        count_bigram[(tag, next_tag)] += 1

  # calculate transition matrix
  f = {}
  q = {}
  for (y_1, y_2), count_bi in count_bigram.items():
    _str_ = "transition:" + y_1 + "+" + y_2
    q[(y_1, y_2)] = np.log(count_bi / count_unigram[y_1])
    if count_bi == 0:
      f[_str_] = - np.inf
    else:
      f[_str_] = np.log(count_bi / count_unigram[y_1])
  return q, f



In [ ]:
e,f = emission("/content/NLP1d/dataset/train")
q,f1 = transition("/content/NLP1d/dataset/train")
f.update(f1)

## Part 2

In [ ]:
def tokenize(file_path):  
    data, lst = [], []
    with open(file_path, 'r') as f:  
        for line in f:
            if line== '\n':
                data.append(lst)
                lst = []
            else:
                lines = line.replace("\n",'').split(" ")
                lst.append(tuple(lines))
    return data

train_sentences = tokenize('/content/NLP1d/dataset/train')

In [ ]:
def wordtags(data):
  all_words, all_tags = [], []
  for lst in data:         
    for wordTuples in lst:
      all_tags.append(wordTuples[1])
      all_words.append(wordTuples[0])
  all_words, all_tags = list(set(all_words)), list(set(all_tags))
  return all_words, all_tags

all_words, all_tags = wordtags(train_sentences)


In [ ]:
def score_sequence(xy,dict_transition,dict_emission):
  x = [x[0] for x in xy]
  y = ['START']+[y[1] for y in xy]+['STOP']
  sum=0
  for i in range(len(x)):
    key = (x[i], y[i+1])
    sum+= dict_emission[key]
  for j in range(1, len(y)):
    key = (y[j-1], y[j])
    if y[j]=='STOP':
      key = (y[j], y[j-1])
    sum+= dict_transition[key]
  return sum

score_sequence(train_sentences[0],q,e)

-82.94749484771204

In [ ]:
def viterbi(sentence, tags, dict_transition, dict_emission):    
  saved_score = [{}]
  path = {}
    
  for tag in tags:
    saved_score[0][tag] = dict_transition.get(('START',tag),0) + dict_emission.get((sentence[0][0],tag),-2147483648)
    path[tag] = [tag]
  #print(saved_score)

  for word_idx in range(1,len(sentence)):
    saved_score.append({})
    path_temp = {}

    for y in tags:
      score_list_temp=[]
      for y0 in tags:
        score_list_temp.append((saved_score[word_idx-1][y0] + dict_transition.get((y0,y),0) + dict_emission.get((sentence[word_idx][0],y),-2147483648), y0))
      (max_score, tag) = max(score_list_temp)
      saved_score[word_idx][y] = max_score
      path_temp[y] = path[tag] + [y]

    path = path_temp
  #print(saved_score)  
  (max_score, tag) = max([(saved_score[len(sentence)-1][y] + dict_transition.get(('STOP',y),0), y) for y in tags])
  #print(path[tag])
  return max_score, path[tag]

In [ ]:
with open("/content/NLP1d/dataset/dev.p2.out", "w") as outfile:
  for sentence in tokenize('/content/NLP1d/dataset/dev.in'):
    predicted_sequence = viterbi(sentence, all_tags, q, e)[1]

    for i in range(len(sentence)):
      line = "{} {}\n".format(sentence[i][0], predicted_sequence[i])
      outfile.write(line)

    outfile.write("\n")